# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
from datasets.caueeg_script import load_caueeg_task_split
from datasets.caueeg_script import make_dataloader
from datasets.pipeline import eeg_collate_fn
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_and_throughput
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import check_accuracy_multicrop_and_throughput
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## List up the models to check accuracy

In [4]:
# model_names = ['cix5o8ut', '76efnucq', '6a1tnz3r']
model_names = [
'3jmkkwbc',
'1nq1zwxk',
'2s1700lg',
'3pg95uf1',
'86s3czvy',
'1o7cz2rs',
'52v2mp7f',
'3t0s53ki',
'1cmzq9xq',
'2z7is3m0',
'13i117jq',
'3cy4eb97',
'ph0mix3b',
'cygczhuk',
'gma3iwco',
'1xhv4ae6',
'dejxbfxu',
'3874otvp',
'2bw00w05',
'2615y2jm',
'2k8xomy6',
'2452kedi',
'3c9rbdxu',
'2h30v12p',
'1242j086',
]
model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['3jmkkwbc',
 '1nq1zwxk',
 '2s1700lg',
 '3pg95uf1',
 '86s3czvy',
 '1o7cz2rs',
 '52v2mp7f',
 '3t0s53ki',
 '1cmzq9xq',
 '2z7is3m0',
 '13i117jq',
 '3cy4eb97',
 'ph0mix3b',
 'cygczhuk',
 'gma3iwco',
 '1xhv4ae6',
 'dejxbfxu',
 '3874otvp',
 '2bw00w05',
 '2615y2jm',
 '2k8xomy6',
 '2452kedi',
 '3c9rbdxu',
 '2h30v12p',
 '1242j086']


In [5]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [6]:
base_repeat = 200
verbose = False

-----

## Load and check accuracy

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    optimizer = ckpt['optimizer_state']
    scheduler = ckpt['scheduler_state']
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']    
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    _, multicrop_val_dataset = load_caueeg_task_split(dataset_path=config['dataset_path'],
                                                      task=config['task'],
                                                      split='val',
                                                      load_event=config['load_event'],
                                                      file_format=config['file_format'],
                                                      transform=config['transform_multicrop'],
                                                      verbose=verbose)
    multicrop_val_loader = DataLoader(multicrop_val_dataset,
                                       batch_size=config['multi_batch_size'],
                                       shuffle=False,
                                       drop_last=False,
                                       num_workers=0,
                                       pin_memory=True,
                                       collate_fn=eeg_collate_fn)
    
    # train accuracy
    train_acc = check_accuracy(model, train_loader, 
                               config['preprocess_test'], config, repeat=repeat)
    model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc

    # Multi-crop val accuracy
    multi_val_acc = check_accuracy_multicrop(model, multicrop_val_loader, 
                                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Validation Accuracy'] = multi_val_acc
    
    
    # Test accuracy
    test_acc = check_accuracy(model, test_loader, 
                       config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Accuracy'] = test_acc

    # Multi-crop test accuracy
    multi_test_acc = check_accuracy_multicrop(model, multicrop_test_loader, 
                                              config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Test Accuracy'] = multi_test_acc

- checking for 3jmkkwbc 1D-ResNet-18 ...
- checking for 1nq1zwxk 1D-ResNet-18 ...
- checking for 2s1700lg 1D-ResNet-18 ...
- checking for 3pg95uf1 1D-ResNet-18 ...
- checking for 86s3czvy 1D-ResNet-18 ...
- checking for 1o7cz2rs 1D-ResNet-18 ...
- checking for 52v2mp7f 1D-ResNet-18 ...
- checking for 3t0s53ki 1D-ResNet-18 ...
- checking for 1cmzq9xq 1D-ResNet-18 ...
- checking for 2z7is3m0 1D-ResNet-18 ...
- checking for 13i117jq 1D-ResNet-18 ...
- checking for 3cy4eb97 1D-ResNet-18 ...
- checking for ph0mix3b 1D-ResNet-18 ...
- checking for cygczhuk 1D-ResNet-18 ...
- checking for gma3iwco 1D-ResNet-18 ...
- checking for 1xhv4ae6 1D-ResNet-18 ...
- checking for dejxbfxu 1D-ResNet-18 ...
- checking for 3874otvp 1D-ResNet-18 ...
- checking for 2bw00w05 1D-ResNet-18 ...
- checking for 2615y2jm 1D-ResNet-18 ...
- checking for 2k8xomy6 1D-ResNet-18 ...
- checking for 2452kedi 1D-ResNet-18 ...
- checking for 3c9rbdxu 1D-ResNet-18 ...
- checking for 2h30v12p 1D-ResNet-18 ...
- checking for 1

In [8]:
pprint.pprint(model_pool)

[{'Multi-Crop Test Accuracy': 61.67796610169491,
  'Multi-Crop Throughput': 1023.7335981381888,
  'Multi-Crop Validation Accuracy': 65.21008403361344,
  'Test Accuracy': 60.101694915254235,
  'Throughput': 8763.763870570338,
  'Train Accuracy': 99.33035714285714,
  'Validation Accuracy': 63.15126050420168,
  'crop_multiple': 4,
  'name': '3jmkkwbc',
  'path': 'E:\\CAUEEG\\checkpoint\\3jmkkwbc\\checkpoint.pt',
  'repeat': 50,
  'test_crop_multiple': 8},
 {'Multi-Crop Test Accuracy': 60.52542372881356,
  'Multi-Crop Throughput': 1248.6612430999946,
  'Multi-Crop Validation Accuracy': 63.714285714285715,
  'Test Accuracy': 59.79237288135593,
  'Throughput': 10922.74345756435,
  'Train Accuracy': 99.98549107142857,
  'Validation Accuracy': 61.65546218487395,
  'crop_multiple': 4,
  'name': '1nq1zwxk',
  'path': 'E:\\CAUEEG\\checkpoint\\1nq1zwxk\\checkpoint.pt',
  'repeat': 50,
  'test_crop_multiple': 8},
 {'Multi-Crop Test Accuracy': 68.33898305084746,
  'Multi-Crop Throughput': 1224.24080

In [9]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
        
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    # torch.save(model_state, 'temptemptemp.pt')
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    
    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')

In [10]:
pd.DataFrame(model_pool)

In [11]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-task2-ablations-summary.csv')